In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import os, pickle, shutil, random, PIL
from PIL import Image

import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader,random_split,Dataset, ConcatDataset ,SubsetRandomSampler
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.transforms import v2

import matplotlib.pyplot as plt
# from torchinfo import summary
from training_utils import *

In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

DFNAME = 'MvNM_rn_34'
device = torch.device('cuda:0')
criterion = nn.CrossEntropyLoss()

modelname = 'MvNM_rn_34'
n_epochs = 5
batch_size = 4

In [ ]:
train_dir = 'malignant_benign_1024/train/'
test_dir = 'malignant_benign_1024/test/'

In [ ]:
size = (1024,768)
train_set_whole = ImageFolder(train_dir,transform = transforms.Compose([
    v2.Resize(size),
    v2.RandomHorizontalFlip(0.5),
    v2.RandomVerticalFlip(0.5),
    v2.RandomRotation(30),
    v2.ToTensor(),
]))

test_set = ImageFolder(test_dir,transform = transforms.Compose([
    v2.Resize(size),
    v2.ToTensor(),
    ]))

train_set, valid_set = random_split(train_set_whole,[int(len(train_set_whole)*0.9), int(len(train_set_whole)*0.1)],
                                  generator=torch.Generator().manual_seed(0))

In [ ]:
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers = 4)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers = 4)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers = 4)
test_loader_2 = DataLoader(test_set, batch_size=1, shuffle=False)

In [ ]:
### Use any one of the following three models. Comment out the ones you don't use or don't run it.

In [ ]:
model = models.resnet34().to(device)
model.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=512,
                    out_features=2, # same number of output units as our number of classes
                    bias=True)).to(device)

In [ ]:
model = models.efficientnet_b0().to(device)
model.classifier = torch.nn.Sequential(
    torch.nn.Linear(in_features=1280,
                    out_features=2, # same number of output units as our number of classes
                    bias=True)).to(device)

In [ ]:
model = models.densenet121().to(device)
model.classifier = torch.nn.Sequential(
    torch.nn.Linear(in_features=1024,
                    out_features=2, # same number of output units as our number of classes
                    bias=True)).to(device)

In [ ]:
optim = torch.optim.SGD(model.parameters(),lr=0.0005, weight_decay=1e-4)

In [ ]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, mode='min', factor=0.5, patience=3)

history = {'train_loss': [], 'valid_loss': [],'train_acc':[],'valid_acc':[]}

for epoch in range(n_epochs):

    train_loss, train_acc = train(model,train_loader,criterion,optim,device,epoch)
    valid_loss, valid_acc = test(model,valid_loader,criterion,optim,modelname,device,epoch)

    scheduler.step(valid_loss)

    history['train_loss'].append(train_loss)
    history['valid_loss'].append(valid_loss)
    history['train_acc'].append(train_acc)
    history['valid_acc'].append(valid_acc)


with open('./content/storage/' + DFNAME + '.pkl', 'wb') as f:
    pickle.dump(history, f)

In [ ]:
### While testing use the same model that was used for training. We are creating a new instance of that model
### with the best performing weights that we stored during training

In [ ]:
new_model = models.resnet34().to(device)
new_model.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=512,
                    out_features=2, # same number of output units as our number of classes
                    bias=True)).to(device)

In [ ]:
new_model = models.efficientnet_b0().to(device)
new_model.classifier = torch.nn.Sequential(
    torch.nn.Linear(in_features=1280,
                    out_features=2, # same number of output units as our number of classes
                    bias=True)).to(device)

In [ ]:
new_model = models.densenet121().to(device)
new_model.classifier = torch.nn.Sequential(
    torch.nn.Linear(in_features=1024,
                    out_features=2, # same number of output units as our number of classes
                    bias=True)).to(device)

In [ ]:
checkpoint = torch.load('./checkpoint/'+modelname+'model.pth.tar',map_location=torch.device('cpu'))
new_model.load_state_dict(checkpoint['model'])

_, _, y, y_pred = best_test(new_model,test_loader,criterion,optim,device,0)